GeoViews is designed to make full use of data declared in the Iris data format, called "cubes".  This notebook shows how to use data in this format. The cubes used in this notebook are publicly available in the [``SciTools/iris-sample-data``](https://github.com/SciTools/iris-sample-data) repository.

In [ ]:
import datetime
import iris
import numpy as np
import holoviews as hv
import geoviews as gv
from cartopy import crs
from cartopy import feature as cf
hv.notebook_extension()

## Setting some notebook-wide options

Let's start by setting some normalization options (discussed below) and always enable colorbars for the elements we will be displaying:

In [ ]:
iris.FUTURE.strict_grib_load = True
%opts Image {+framewise} [colorbar=True] {+framewise} Curve [xrotation=60]

Note that it is easy to set global defaults for a project, allowing any suitable settings to be made into a default on a per-element-type basis. Now let's specify the maximum number of frames we will be displaying:

In [ ]:
%output max_frames=1000 


<div class="alert alert-info" role="alert">When working on a live server append ``widgets='live'`` to the line above for greatly improved performance and memory usage </div>


## Loading our first cube

Here is the summary of the first cube, which contains some surface temperature data:

In [ ]:
iris_cube = iris.load_cube(iris.sample_data_path('GloSea4', 'ensemble_001.pp'))
iris_cube.coord('latitude').guess_bounds()
iris_cube.coord('longitude').guess_bounds()

In [ ]:
print iris_cube.summary()

Now we can wrap this Iris cube in a ``Dataset``:

In [ ]:
surface_temperature = hv.Dataset(iris_cube)
surface_temperature

As you can see, this `Dataset` is not yet visualizable, because we have not chosen which dimensions to map onto which axes of a plot.

# A Simple example

To visualize the ``surface_temperature`` cube, in a single line of code we can specify that we want to view it as a collection of Images indexed by longitude and latitude (a HoloViews ``HoloMap`` of ``gv.Image`` elements):

In [ ]:
surface_temperature.to(gv.Image,['longitude', 'latitude'])

You can see that the `time` dimension was automatically mapped to a slider, because we did not map it onto one of the other available dimensions (x, y, or color, in this case). You can drag the slider to view the surface temperature at different times. The available `time` values can be shown by using the HoloViews API:

In [ ]:
surface_temperature.dimension_values('time')

The human-readable times shown in the slider are long, making the text rather small. We can use the fact that all times are recorded in the year 2011 on the 16th of each month to shorten these dates. Defining how all dates should be formatted as follows will help with readability:

In [ ]:
hv.Dimension.type_formatters[datetime.datetime] = "%m/%y %Hh"

Now let us load a cube showing the pre-industrial air temperature:

In [ ]:
air_temperature = hv.Dataset(iris.load_cube(iris.sample_data_path('pre-industrial.pp')),
                             group='Pre-industrial air temperature')
air_temperature.data.coord('longitude').guess_bounds()
air_temperature.data.coord('latitude').guess_bounds()
air_temperature     # Use air_temperature.data.summary() to see the Iris summary (.data is the Iris cube)

Note that we have the ``air_temperature`` available over ``longitude`` and ``latitude`` but *not* the ``time`` dimensions. As a result, this cube is a single frame (at right below) when visualized as a temperature map.

In [ ]:
%%opts Layout [fig_inches=(12,4)]
(surface_temperature.to.image(['longitude', 'latitude'])+
 air_temperature.to.image(['longitude', 'latitude'])(plot=dict(projection=crs.PlateCarree())))

Next is a fairly involved example that plots an arbitrary number of HoloViews objects side by side in a ``Layout``, rather than using the fixed-layout ``+`` operator. 

This example shows how complex interactive plots can be generated with relatively little code, and also demonstrates how different HoloViews elements can be combined together. In the following visualization, the curve is a sample of the ``surface_temperature`` at longitude and latitude *(0,10)*, and it is unaffected by the `time` slider because it already lays out time along the x axis:

In [ ]:
%%opts Layout [fig_inches=(12,7)] Curve [aspect=2 xticks=4 xrotation=15] Points (color=2)
%%opts Image [projection=crs.PlateCarree()]

temp_curve = hv.Curve(surface_temperature.select(longitude=0, latitude=10), kdims=['time'])

temp_maps = [cb.to(gv.Image,['longitude', 'latitude']) * gv.Points([(0,10)]) 
             for cb in [surface_temperature, air_temperature]]

hv.Layout(temp_maps + [temp_curve]).cols(2).display('all')

## Overlaying data and normalization

Let's view the surface temperatures together with the global coastline:

In [ ]:
%%opts Image [projection=crs.Geostationary()] (cmap='Greens')
surface_temperature.to.image(['longitude', 'latitude']) * gv.Feature(cf.COASTLINE)

Notice that every frame individually uses the full dynamic range of the Greens color map. This is because normalization is set to ``+framewise`` at the top of the notebook, which means every frame is normalized independently. This sort of normalization can be computed on an as-needed basis, using whatever values are found in the current data being shown in a given frame, but it won't let you see how different frames compare to each other.

To control normalization, we need to decide on the normalization limits. Let's see the maximum temperature in the cube, and use it to set a normalization range by declaring a value dimension for the Dataset:

In [ ]:
max_surface_temp = surface_temperature.data.data.max()
max_surface_temp

In [ ]:
%%opts Image [projection=crs.Geostationary()] (cmap='Greens')
surface_temp_dim = hv.Dimension('surface_temperature', range=(300, max_surface_temp))
hv.Dataset(surface_temperature, vdims=[surface_temp_dim]).to(gv.Image,['longitude', 'latitude']) \
  * gv.Feature(cf.COASTLINE)

By specifying the normalization range we can reveal different aspects of the data. In the example above we can see a cooling effect over time as the dark green areas close to the top of the normalization range (317K) vanish. Values outside this range are clipped to the ends of the color map.

Lastly, here is a demo of a conversion from ``surface_temperature`` to ``FilledContours``:

In [ ]:
surface_temperature.to(gv.FilledContours,['longitude', 'latitude']) * gv.Feature(cf.COASTLINE)

As you can see, it's quite simple to expose any data you like from your Iris cube, easily and flexibly creating interactive or static visualizations.